In [1]:
# Import necessary libraries
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

import scenarios as sce

ModuleNotFoundError: No module named 'scenarios'

In [ ]:
model_inputs_path = '../../model/inputs/'
model_outputs_path = '../../model/outputs/'
gen_per_res_vs_marg_cost = '../../data/XM-API/Plan/gen_per_res_vs_marg_cost/Esc0.csv'
emissions = '../../data/XM-API/Plan/emissions/esc0.csv'
cap_2023 = '../../data/XM-API/Plan/installed_capacity/esc0/2023.csv'
cap_2037 = '../../data/XM-API/Plan/installed_capacity/esc0/2037.csv'
dema_path = '../../data/XM-API/variable_query/2022-12-01_2023-11-30/'

In [4]:
import scenarios as sce
sce.table(model_outputs_path, model_inputs_path)

Year,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037
gen_tech,,,,,,,,,,,,,,,
Eolica,20.00,0.0,0.0,0.0,255.0,450.0,492.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Hidro,45.64,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Thermal,0.00,52.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
pv_solar,466.60,1079.9,110.0,218.0,128.0,128.0,123.0,109.0,109.0,91.0,79.0,80.0,68.0,51.0,69.0
Total,532.24,1131.9,110.0,218.0,383.0,578.0,615.0,109.0,109.0,91.0,79.0,80.0,68.0,51.0,69.0


In [5]:
sce_sw = pd.read_csv(model_outputs_path+'/dispatch.csv')
sce_sw['year'] = sce_sw['timestamp'].str[:4]
sce_sw['Energy_TWh_typical_yr'] = sce_sw['Energy_GWh_typical_yr'] / 1000
sce.dispatched_generation(sce_sw, 'year', 'Energy_TWh_typical_yr', 'gen_tech')

In [6]:
sce_xm = pd.read_csv(gen_per_res_vs_marg_cost)
sce_xm['Energy_TWh_typical_yr'] = sce_xm['Energy_GWh_typical_yr'] / 1000
sce.dispatched_generation(sce_xm, 'year', 'Energy_TWh_typical_yr', 'Tech')

In [7]:
em_sw = pd.read_csv(model_outputs_path+'/emissions.csv')
em_sw['AnnualEmissions_MtCO2_per_yr'] = em_sw['AnnualEmissions_tCO2_per_yr'] / 1000000
sce.annual_emmissions(em_sw, 'PERIOD', 'AnnualEmissions_MtCO2_per_yr')

In [8]:
em_xm = pd.read_csv(emissions)
sce.annual_emmissions(em_xm, 'PERIOD', 'AnnualEmissions_tCO2_per_yr')

In [13]:
cap_inst = pd.read_csv(model_outputs_path+'BuildGen.csv')
gen_info = pd.read_csv(model_inputs_path+'gen_info.csv')
cap_inst = pd.merge(cap_inst, gen_info, left_on='GEN_BLD_YRS_1',
    right_on='GENERATION_PROJECT', how='inner')
## Remove unactive plants
cap_inst = cap_inst[cap_inst['gen_max_age'] > 1]

cap_inst = cap_inst[['GENERATION_PROJECT','gen_tech','GEN_BLD_YRS_2','BuildGen']]
cap_inst_2023 = cap_inst[cap_inst['GEN_BLD_YRS_2'] <= 2023]
cap_inst_2023 = cap_inst_2023.groupby(['gen_tech']).agg({
    'BuildGen' : 'sum'}).reset_index()

cap_inst = cap_inst[cap_inst['GEN_BLD_YRS_2'] <= 2037]
cap_inst = cap_inst.groupby(['gen_tech']).agg({
    'BuildGen' : 'sum'}).reset_index()

sce.installed_capacity(cap_inst_2023, cap_inst)

In [ ]:
import pandas as pd
esc_cap_2023 = pd.read_csv(cap_2023)
esc_cap_2037 = pd.read_csv(cap_2023)

sce.installed_capacity(esc_cap_2023, esc_cap_2037)